# 00: Collect data 

This assums that you have downloaded projects data file from the [RePORTER](https://reporter.nih.gov/exporter/projects) website. Make sure that there is a data folder in the project root folder in order for this notebook to work. 

Check list:
- Make sure you have a `data` folder in root project folder
- Make sure that you have downloaded the RePORTER dataset and it is stored within the project root folder

This notebook will merge all dataset and will output a compressed one. 

In [1]:
from pathlib import Path
import pandas as pd

## Parameters
Here are the parameters used to run this notebook:
- `PREFIX`: {str} -> unique wildcard used to identify all files
- `DATA_PATH`: {str} -> path to where all the RePORTER files are stored
- `OUTDIR`: {str} -> Path to results directory
- `OUTNAME`: {str} -> output name of the generated merged file

In [2]:
PREFIX = "RePORTER"
DATA_PATH = "../data"
OUTDIR = "../results"
OUTNAME = "projects_2019-2022"

## Getting data file paths

In [3]:
# creating out directory
out_dir_path = Path(OUTDIR)
out_dir_path.mkdir(exist_ok=True)

# loading in the RePORTER dataset
data_path = Path(DATA_PATH)
all_files = [str(_path.absolute()) for _path in data_path.glob(f"{PREFIX}_*.csv")]
all_files

['/home/erikserrano/Development/CytoData-Faculty-Search/notebooks/../data/RePORTER_PRJ_C_FY2021.csv',
 '/home/erikserrano/Development/CytoData-Faculty-Search/notebooks/../data/RePORTER_PRJ_C_FY2020.csv',
 '/home/erikserrano/Development/CytoData-Faculty-Search/notebooks/../data/RePORTER_PRJ_C_FY2019_new.csv']

## Loading data files into a pandas dataframe

The formatting of these csv files contains some lines that causes pandas tokenizer to fail
- we encode the lines, the `ignore` will remove any invalid characters that are not utf-8
- on_bad_lines is set to skip in order to prevent the C tokenization function from failing
- project_df contains all the project information from multiple files.
- this will store all dataframes and will be concatenated into a single cone

In [4]:
# The formatting of these csv files contains some lines that causes pandas tokenizer to fail
# -- we encode the lines, the `ignore` will remove any invalid characters that are not utf-8
# -- on_bad_lines is set to skip in order to prevent the C tokenization function from failing

# -- project_df contains all the project information from multiple files.
# -- this will store all dataframes and will be concatenated into a single cone
project_df = []
for _file in all_files:
    df = pd.read_csv(
        _file,
        encoding="utf-8",
        encoding_errors="ignore",
        on_bad_lines="skip",
    )
    project_df.append(df)

project_df = pd.concat(project_df)

# printing out df metadata
rows, columns = project_df.shape
print(f"MESSAGE: Dataframe loaded {rows} rows and {columns} columns")
print("WARNING: Some entries may be omitted due to content not being utf-8 compatible")
project_df.head()

/tmp/ipykernel_592613/2703979073.py:9: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


MESSAGE: Dataframe loaded 244429 rows and 46 columns


,APPLICATION_ID,ACTIVITY,ADMINISTERING_IC,APPLICATION_TYPE,ARRA_FUNDED,AWARD_NOTICE_DATE,BUDGET_START,BUDGET_END,CFDA_CODE,CORE_PROJECT_NUM,...,SERIAL_NUMBER,STUDY_SECTION,STUDY_SECTION_NAME,SUBPROJECT_ID,SUFFIX,SUPPORT_YEAR,DIRECT_COST_AMT,INDIRECT_COST_AMT,TOTAL_COST,TOTAL_COST_SUB_PROJECT
0,10595864,U54,DK,6.0,N,06/13/2022,04/01/2022,07/31/2022,NaN,U54DK106829,...,106829.0,ZDK1,Special Emphasis Panel,7612.0,NaN,7.0,42060.0,31955.0,NaN,74015.0
1,10101643,R01,DA,5.0,N,02/22/2021,03/01/2021,02/28/2022,279.0,R01DA046197,...,46197.0,ZRG1,Special Emphasis Panel,NaN,NaN,4.0,451257.0,167187.0,618444.0,NaN
2,10189622,U18,FD,5.0,N,06/08/2021,06/01/2021,05/31/2022,103.0,U18FD006442,...,6442.0,ZFD1,Special Emphasis Panel,NaN,NaN,4.0,NaN,NaN,74000.0,NaN
3,10189608,U18,FD,5.0,N,06/01/2021,06/01/2021,05/31/2022,103.0,U18FD006164,...,6164.0,ZFD1,Special Emphasis Panel,NaN,NaN,5.0,NaN,NaN,52000.0,NaN
4,10076833,R01,EY,5.0,N,01/11/2021,01/01/2021,12/31/2021,867.0,R01EY015240,...,15240.0,BVS,Biology of the Visual System Study Section,NaN,NaN,16.0,335775.0,204822.0,540597.0,NaN


In [5]:
# These are the given features along with metadata
project_df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244429 entries, 0 to 79328
Data columns (total 46 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   APPLICATION_ID          244429 non-null  int64  
 1   ACTIVITY                244429 non-null  object 
 2   ADMINISTERING_IC        244429 non-null  object 
 3   APPLICATION_TYPE        240173 non-null  float64
 4   ARRA_FUNDED             244429 non-null  object 
 5   AWARD_NOTICE_DATE       230704 non-null  object 
 6   BUDGET_START            230760 non-null  object 
 7   BUDGET_END              230734 non-null  object 
 8   CFDA_CODE               194806 non-null  float64
 9   CORE_PROJECT_NUM        240538 non-null  object 
 10  ED_INST_TYPE            145283 non-null  object 
 11  FOA_NUMBER              230552 non-null  object 
 12  FULL_PROJECT_NUM        244429 non-null  object 
 13  FUNDING_ICs             237849 non-null  object 
 14  FUNDING_MECHANISM    

## Save combined file into compressed gz file

In [6]:
save_path = out_dir_path / f"{OUTNAME}.csv.gz"
project_df.to_csv(save_path, index=False, compression="gzip")